In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [5]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
X = np.array(df.iloc[:, 1:])
y = to_categorical(np.array(df.iloc[:, 0]))

# Convert the training and test images into 3 channels
X = np.dstack([X] * 3)
# Reshape images as per the tensor format required by tensorflow
X = X.reshape(-1, 28,28,3)
X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in X])
X = X.astype('float32')/255

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
for fc in [1024, 512, 256]:
    x = Dense(fc, activation='relu')(x)
    x = Dropout(dropout)(x)
# x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=30, verbose=1, validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 49s 1ms/sample - loss: 1.2208 - accuracy: 0.5310 - val_loss: 1.8553 - val_accuracy: 0.4417
Epoch 2/50
48000/48000 [==============================] - 42s 873us/sample - loss: 0.8428 - accuracy: 0.6455 - val_loss: 2.2835 - val_accuracy: 0.4398
Epoch 3/50
48000/48000 [==============================] - 42s 870us/sample - loss: 0.7434 - accuracy: 0.6875 - val_loss: 3.7044 - val_accuracy: 0.3965
Epoch 4/50
48000/48000 [==============================] - 41s 855us/sample - loss: 0.6751 - accuracy: 0.7222 - val_loss: 3.1960 - val_accuracy: 0.4030
Epoch 5/50
48000/48000 [==============================] - 41s 857us/sample - loss: 0.6124 - accuracy: 0.7477 - val_loss: 3.6605 - val_accuracy: 0.3892
Epoch 6/50
48000/48000 [==============================] - 41s 854us/sample - loss: 0.5573 - accuracy: 0.7761 - val_loss: 3.8451 - val_accuracy: 0.4103
Epoch 7/50
47872/48000 [======================

In [8]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_2
1 conv2d_94
2 batch_normalization_94
3 activation_94
4 conv2d_95
5 batch_normalization_95
6 activation_95
7 conv2d_96
8 batch_normalization_96
9 activation_96
10 max_pooling2d_4
11 conv2d_97
12 batch_normalization_97
13 activation_97
14 conv2d_98
15 batch_normalization_98
16 activation_98
17 max_pooling2d_5
18 conv2d_102
19 batch_normalization_102
20 activation_102
21 conv2d_100
22 conv2d_103
23 batch_normalization_100
24 batch_normalization_103
25 activation_100
26 activation_103
27 average_pooling2d_9
28 conv2d_99
29 conv2d_101
30 conv2d_104
31 conv2d_105
32 batch_normalization_99
33 batch_normalization_101
34 batch_normalization_104
35 batch_normalization_105
36 activation_99
37 activation_101
38 activation_104
39 activation_105
40 mixed0
41 conv2d_109
42 batch_normalization_109
43 activation_109
44 conv2d_107
45 conv2d_110
46 batch_normalization_107
47 batch_normalization_110
48 activation_107
49 activation_110
50 average_pooling2d_10
51 conv2d_106
52 conv2d_108
53 conv2d

In [9]:
for layer in model.layers[:288]:
   layer.trainable = False
for layer in model.layers[288:]:
   layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=30, verbose=1, validation_data=(X_val, y_val))

Train on 48000 samples, validate on 6000 samples
Epoch 1/50
48000/48000 [==============================] - 49s 1ms/sample - loss: 0.0762 - val_loss: 15.1119
Epoch 2/50
48000/48000 [==============================] - 42s 879us/sample - loss: 0.0442 - val_loss: 14.9987
Epoch 3/50
48000/48000 [==============================] - 43s 900us/sample - loss: 0.0387 - val_loss: 15.0123
Epoch 4/50
48000/48000 [==============================] - 45s 929us/sample - loss: 0.0339 - val_loss: 14.9877
Epoch 5/50
48000/48000 [==============================] - 42s 882us/sample - loss: 0.0293 - val_loss: 15.0370
Epoch 6/50
48000/48000 [==============================] - 44s 913us/sample - loss: 0.0274 - val_loss: 14.6454
Epoch 7/50
48000/48000 [==============================] - 44s 927us/sample - loss: 0.0244 - val_loss: 14.8529
Epoch 8/50
48000/48000 [==============================] - 43s 889us/sample - loss: 0.0254 - val_loss: 14.6497
Epoch 9/50
48000/48000 [==============================] - 44s 919us/sampl

In [11]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 14.827917289733886
